In [ ]:
# 使用 Redis 来缓存提示和响应，并根据语义相似度评估命中情况。
# , n=2, best_of=2
# n：针对每个提示要生成的聊天完成次数。


In [3]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

cf_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('CF_API_BASE'),
    openai_api_key=os.getenv('CF_API_TOKEN'),
    model_name="@cf/meta/llama-3-8b-instruct",
    temperature=0,
    streaming=True
)

In [13]:
qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)

In [15]:
groq_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('GROQ_API_BASE'),
    openai_api_key=os.getenv('GROQ_API_KEY'),
    model_name="llama3-8b-8192",
    temperature=0,
    streaming=True,
)

In [17]:
ms_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('MOONSHOT_API_BASE'),
    openai_api_key=os.getenv('MOONSHOT_API_KEY'),
    model_name="moonshot-v1-8k",
    temperature=0,
    streaming=True,
)

In [8]:
from langchain_community.embeddings.cloudflare_workersai import CloudflareWorkersAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()
embedding = CloudflareWorkersAIEmbeddings(
    account_id=os.getenv('CF_ACCOUNT_ID'),
    api_token=os.getenv('CF_API_TOKEN'),
    model_name="@cf/baai/bge-small-en-v1.5",
)

In [21]:
# from langchain.embeddings import OpenAIEmbeddings
from langchain_community.cache import RedisSemanticCache
import langchain

REDIS_URL = "redis://192.168.22.238:6379/0"

langchain.llm_cache = RedisSemanticCache(redis_url="redis://192.168.22.238:6379",
                                         embedding=embedding,
                                         score_threshold=0.2)

In [23]:
from langchain.globals import set_llm_cache

set_llm_cache(RedisSemanticCache(redis_url="redis://192.168.22.238:6379",
                                 embedding=embedding,
                                 score_threshold=0.2))

In [24]:
# %%time

ms_llm_openai.invoke("Please translate 'this is Monday' into Chinese")

ResponseError: unknown command 'module'

In [27]:
langchain.llm_cache = RedisSemanticCache(redis_url="redis://192.168.22.238:6379", 
                                         embedding=embedding, score_threshold=0.05)


In [28]:
qw_llm_openai.invoke("Please translate 'this is Monday' into Chinese")


ResponseError: unknown command 'module'

In [ ]:
# 深入研究 Redis 语义缓存

In [29]:
langchain.llm_cache._cache_dict

{}

In [ ]:
langchain.llm_cache._cache_dict['cache:bf6f6d9ebdf492e28cb8bf4878a4b951'].similarity_search_with_score(query='Give me 2 peaches')
